<a href="https://colab.research.google.com/github/KamonohashiPerry/kaggle/blob/master/Titanic_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
# files.upload()

In [0]:
pwd

'/content'

In [0]:
ls

adc.json  gender_submission.csv  kaggle.json  sample_data/  test.csv  train.csv


In [0]:
# !mkdir -p ~/.kaggle

In [0]:
# !mv kaggle.json ~/.kaggle/

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename =  "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!pip install kaggle

In [0]:
!kaggle competitions list

ref                                            deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                               2030-01-01 00:00:00  Getting Started   Knowledge       2209           False  
titanic                                        2030-01-01 00:00:00  Getting Started   Knowledge      16425            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started   Knowledge       5424            True  
imagenet-object-localization-challenge         2029-12-31 07:00:00  Research          Knowledge         61           False  
competitive-data-science-predict-future-sales  2020-12-31 23:59:00  Playground            Kudos       5308            True  
deepfake-detection-challenge                   2020-03-31 23:59:00  Featured         $1,000,000        791           False  


In [0]:
!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
ls

adc.json  gender_submission.csv  kaggle.json  sample_data/  test.csv  train.csv


In [0]:
import pandas as pd

In [0]:
# 学習データ、テストデータの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
import pandas_profiling as pdp

In [0]:
pdp.ProfileReport(train)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,12
Number of observations,891
Total Missing (%),8.1%
Total size in memory,83.7 KiB
Average record size in memory,96.1 B
Numeric,6
Categorical,4
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [0]:
# テストデータは特徴量のみなので、そのまま
test_x = test.copy()

In [0]:
# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']

from sklearn.preprocessing import LabelEncoder
# 変数PassengerIdを除外する
train_x = train_x.drop(['PassengerId'], axis=1)
test_x = test_x.drop(['PassengerId'], axis=1)

# 変数Name, Ticket, Cabinを除外する
train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# それぞれのカテゴリ変数にlabel encodingを適用する
for c in ['Sex', 'Embarked']:
  # 学習データに基づいてどう変換するかを定める
  le = LabelEncoder()
  le.fit(train_x[c].fillna('NA'))

  # 学習データ、テストデータを変換する
  train_x[c] = le.transform(train_x[c].fillna('NA'))
  test_x[c] = le.transform(test_x[c].fillna('NA'))


In [0]:
from xgboost import XGBClassifier
import numpy as np

# モデルの作成および学習データを与えての学習
model = XGBClassifier(n_estimators=20,
                       random_state=71)
model.fit(train_x, train_y)

# テストデータの予測値を確率で出力する
pred = model.predict_proba(test_x)[:, 1]

# テストデータの予測値を二値に変換する
pred_label = np.where(pred > 0.5, 1, 0)

# 提出用ファイルの作成
submission = pd.DataFrame({'PassengerId':test['PassengerId'],
                             'Survived':pred_label})
submission.to_csv('submission_first.csv', index=False)

In [0]:
ls

adc.json               kaggle.json   submission_first.csv  train.csv
gender_submission.csv  sample_data/  test.csv


In [0]:
# ファイルのダウンロード
 files.download("submission_first.csv")

In [0]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

# 各foldのスコアを保存するリスト
scores_accuracy = []
scores_logloss = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションとすることを、バリデーションデータを変える繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
  # 学習データを学習データとバリデーションデータに分ける
  tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
  tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

  # モデルの学習を行う
  model = XGBClassifier(n_estimators=20, random_state=71)
  model.fit(tr_x, tr_y)

  # バリデーションデータの予測値を確率で出力する
  va_pred = model.predict_proba(va_x)[:, 1]

  # バリデーションデータでのスコアを計算する
  logloss = log_loss(va_y, va_pred)
  accuracy = accuracy_score(va_y, va_pred > 0.5)

  # そのfoldのスコアを保存する
  scores_logloss.append(logloss)
  scores_accuracy.append(accuracy)

  # 各foldのスコアの平均を出力する
  logloss = np.mean(scores_logloss)
  accuracy = np.mean(scores_accuracy)
  print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')

logloss: 0.3972, accuracy: 0.8341
logloss: 0.4155, accuracy: 0.8206
logloss: 0.4265, accuracy: 0.8146
logloss: 0.4270, accuracy: 0.8148


In [0]:
# -----------------------------------
# ロジスティック回帰用の特徴量の作成
# -----------------------------------
from sklearn.preprocessing import OneHotEncoder

# 元データをコピーする
train_x2 = train.drop(['Survived'], axis=1)
test_x2 = test.copy()

# 変数PassengerIdを除外する
train_x2 = train_x2.drop(['PassengerId'], axis=1)
test_x2 = test_x2.drop(['PassengerId'], axis=1)

# 変数Name, Ticket, Cabinを除外する
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# one-hot encodingを行う
cat_cols = ['Sex', 'Embarked', 'Pclass']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x2[cat_cols].fillna('NA'))

# one-hot encodingのダミー変数の列名を作成する
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]

# one-hot encodingによる変換を行う
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns=ohe_columns)

# one-hot encoding済みの変数を除外する
train_x2 = train_x2.drop(cat_cols, axis=1)
test_x2 = test_x2.drop(cat_cols, axis=1)

# one-hot encodingで変換された変数を結合する
train_x2 = pd.concat([train_x2, ohe_train_x2], axis=1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis=1)

# 数値変数の欠損値を学習データの平均で埋める
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(), inplace=True)
    test_x2[col].fillna(train_x2[col].mean(), inplace=True)

# 変数Fareを対数変換する
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])

In [0]:
import itertools

# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth':[3, 5, 7],
    'min_child_weight':[1.0, 2.0, 4.0]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:
  score_folds = []

  # クロスバリデーションを行う
  # 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
  kf = KFold(n_splits=4,shuffle=True,random_state=123456)
  for tr_idx, va_udx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71,
                          max_depth=max_depth,
                          min_child_weight=min_child_weight)
    model.fit(tr_x, tr_y)

    # バリデーションデータでのスコアを計算し、保存する
    va_pred = model.predict_proba(va_x)[:, 1]
    logloss = log_loss(va_y, va_pred)
    score_folds.append(logloss)

  # 各foldのスコアを平均する
  score_mean = np.mean(score_folds)

  # パラメータの組み合わせ、それに対するスコアを保存する
  params.append((max_depth, min_child_weight))
  scores.append(score_mean)

# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

# max_depth=7, min_child_weight=2.0のスコアが最も良かった
from sklearn.linear_model import LogisticRegression

# xgboostモデル
model_xgb = XGBClassifier(n_estimators=20, random_state=71)
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

# ロジスティック回帰モデル
# xgboostモデルとは異なる特徴量を入れる必要があるので、別途train_x2, test_x2を作成した
model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
model_lr.fit(train_x2, train_y)
pred_lr = model_lr.predict_proba(test_x2)[:, 1]

# 予測値の加重平均をとる
pred = pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5, 1, 0)

max_depth: 7, min_child_weight: 1.0


## Reference
https://github.com/ghmagazine/kagglebook